In [1]:
import mysql.connector
import os
import sys
import numpy as np

In [2]:
def connect_server():
    try:
        connection = mysql.connector.connect(option_files="connection.ini")
    except mysql.connector.Error as err:
        print(err.errno)
    return connection

In [3]:
def disconnect_server(connection):
    if connection.is_connected():
        connection.close()

In [4]:
def use_database(database,connection,cursor=None):
    needcursor = cursor == None
   
    try:
        if needcursor:
            cursor = connection.cursor()
            cursor.execute(f"USE {database}")
            connection.commit()
        if needcursor:
            cursor.close()
    
    except mysql.connector.Error as err:
        print(err.errno)

<br><h3>Fetch article from database<br><br>

In [5]:
def count_tran_article_entries ():
    connection = connect_server()
    cursor = connection.cursor()
    use_database("automated_journalist",connection)
    
    query = ('SELECT idtran_article FROM tran_article')
    cursor.execute(query)
    result = cursor.fetchall()
    cursor.close()
    disconnect_server(connection)
    return len(result)

In [6]:
def count_orig_article_entries ():
    connection = connect_server()
    cursor = connection.cursor()
    use_database("automated_journalist",connection)
    
    query = ('SELECT idorig_article FROM orig_article')
    cursor.execute(query)
    result = cursor.fetchall()
    cursor.close()
    disconnect_server(connection)
    return len(result)

In [7]:
def retrieve_article (idorig_article):
    connection = connect_server()
    cursor = connection.cursor()
    use_database("automated_journalist",connection)
    
    query = ('SELECT orig_article_text FROM orig_article WHERE idorig_article = %s')
    val = ([idorig_article])
    cursor.execute(query,val)
    result = cursor.fetchall()
    cursor.close()
    disconnect_server(connection)
    return result[0][0]

<br><h3>Remodel through API<br>

In [16]:
import os
import openai
openai.api_key = open("oai.txt", "r").read()
from datetime import date

In [33]:
def convert_article():
    completion =openai.ChatCompletion.create(
        model="gpt-3.5-turbo", 
        messages=[{"role":"system",
               "content":"You are a journalist in the organic foods sector, working for an organic trade journal in europe. Your job is to convert press releases from various public agencies and companies into short journalistic texts of 100 to 200 words and translate them into english. Articles must always quote the source of the information.The primary objective is to inform professionals, primarily management, in the organic industry about current developments in the industry, information about upcoming or past events or new products and services. If aplicable, use information about Who, What, When, Where, Why.  Adhere to the principles of Simplicity and Brevity, Precision, Objectivity and Factual, Fairness and Balance and use the Inverted pyramid, meaning arrange the information in the article in descending order of importance, or newsworthiness. Consider yourself to be benevolently critical: While emphasising positive developments to promote action and progress, you always look out for challenges and criticism in order to improve your readers' companies and the industry as a whole. The generated article must not indicate, that the original source was a press release and must not contain the original publishing date."},
              {"role":"user","content": "Please transform the following Article: "+orig_article}],
        temperature=0.5)

    tran_article=completion.choices[0].message.content
    return tran_article_list.append(tran_article)


<br><h3>Transfer to Database

In [34]:
def retrieve_idorig_article ():
    connection = connect_server()
    cursor = connection.cursor()
    use_database("automated_journalist",connection)
    
    query = ('SELECT idorig_article FROM orig_article WHERE orig_article_text = %s')
    val = ([orig_article])
    cursor.execute(query,val)
    result = cursor.fetchall()
    cursor.close()
    disconnect_server(connection)
    return result[0][0]

In [35]:
def insert_tran_article(tran_article):  
    connection = connect_server()
    cursor = connection.cursor()
    use_database("automated_journalist",connection)
    add_entry = ("INSERT INTO tran_article (tran_article_text, idorig_article, creation_date) VALUES (%s, %s, %s)")
    data_entry = (tran_article, retrieve_idorig_article(), date.today())
    cursor.execute(add_entry, data_entry)
    connection.commit()
    cursor.close()
    disconnect_server(connection)

In [36]:
while count_tran_article_entries() < count_orig_article_entries ():
    tran_article_list=[]
    orig_article = retrieve_article (count_tran_article_entries()+1)
    convert_article()
    insert_tran_article(tran_article_list[0])

# clears table
connection = connect_server()
cursor = connection.cursor()
use_database("automated_journalist",connection)
cursor.execute("TRUNCATE TABLE tran_article")
connection.commit()
cursor.close()
disconnect_server(connection)

<br><h3>Generate headline

In [9]:
def create_headline():
    completion =openai.ChatCompletion.create(
        model="gpt-3.5-turbo", 
        messages=[{"role":"system",
               "content":"You are a SEO Specialist in the organic foods sector, working for an organic trade journal in europe. Your job is to find the most cathing and engaging Headlines for given articles in 10 words or less. Do not use quotation marks in beginning and end of the headline. Targent group are professionals, primarily management, in the organic industry."},
              {"role":"user","content": "Please find a headline for the following Article: "+tran_article}],
        temperature=0.5)

    title=completion.choices[0].message.content
    return title_list.append(title)

In [10]:
def insert_headline():  
    connection = connect_server()
    cursor = connection.cursor()
    use_database("automated_journalist",connection)
    add_entry = ("UPDATE tran_article SET title = %s WHERE tran_article_text = %s")
    data_entry = (title_list[0].strip('\"'), tran_article)
    cursor.execute(add_entry, data_entry)
    connection.commit()
    cursor.close()
    disconnect_server(connection)


In [11]:
def count_tran_title_entries ():
    connection = connect_server()
    cursor = connection.cursor()
    use_database("automated_journalist",connection)
    
    query = ('SELECT title FROM tran_article WHERE title IS NOT NULL')
    cursor.execute(query)
    result = cursor.fetchall()
    cursor.close()
    disconnect_server(connection)
    return len(result)

In [12]:
def retrieve_tran_article (idtran_article):
    connection = connect_server()
    cursor = connection.cursor()
    use_database("automated_journalist",connection)
    
    query = ('SELECT orig_article_text FROM orig_article WHERE idorig_article = %s')
    val = ([idtran_article])
    cursor.execute(query,val)
    result = cursor.fetchall()
    cursor.close()
    disconnect_server(connection)
    return result[0][0]

46

In [15]:
while count_tran_title_entries () < count_tran_article_entries():
    title_list=[]
    tran_article = retrieve_tran_article(count_tran_title_entries ()+1)
    create_headline()
    insert_headline()

RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 984484ebf851a5931b948cfff55cfc13 in your message.)

In [ ]:
count_tran_article_entries()

In [ ]:
count_tran_title_entries ()